## 1 卷积神经网络

在之前的神经网络学习过程中，使用的都是全连接神经网络，全连接神经网络对识别和预测都有非常好的效果。在之前使用 MNIST 数据集的实践过程中，输入神经网络的是是一幅 28 行 28 列的 784 个像素点的灰度值，但是仅两层神经网络就有十多万个待训练参数（第一层$784\times128个\omega+128个b$，第二层$128\times10个\omega+10个b$，共 101770 个参数）。

在实际项目中，输入神经网络的是具有更高分辨率的彩色图片，使得送入全连接网络的输入特征数特别多，随着隐藏层数的增加，网络规模过大，待优化参数过多，很容易造成过拟合。**为了减少待训练参数，在实际应用时会先对原始图片进行特征提取，再把提取到的特征送给全连接网络**。而卷积计算就是一种有效的提取图像特征的方法。

### 1.1 卷积层

#### 1.1.1 卷积过程

卷积（Convolutional）的计算过程：

(1)一般会用一个正方形的卷积核，按指定步长，在输入特征图上滑动，遍历输入特征图中的每个像素点。滑动过程如下所示：

![](http://img.zishu010.com/ArticleImage/2017/8/d658466e93f4488bbdfd0856bceb7b36.gif)

(2)每移动一个步长，卷积核会与输入特征图出现重合区域，重合区域对应元素相乘、求和再加上偏置项得到输出特征的每一个像素点。如果输入特征是单通道灰度图，那么使用的就是深度为 1 的单通道卷积核。那么计算过程如下：

![](https://img-blog.csdnimg.cn/20191106143628622.JPG?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80NTY4MDAwNw==,size_16,color_FFFFFF,t_70)

**单通道与卷积核深度为 1 有什么关系呢**？首先要知道图片的色彩是由 RGB 三原色组成的，也就是说，除去图片的长度和宽度，还有一个表示图片色彩通道（channel）数的“深度”。因此在卷积过程中，一定要保持卷积核的通道数与输入特征图的通道数一致，即**输入特征图的深度决定了当前层卷积核的深度**。那么对于单通道的特征图而言，通道数为 1 即深度为 1，自然卷积核的深度也就是 1。

综上所述，如果输入特征是三通道彩色图，则需要使用一个 $3\times3\times3$ 的卷积核，结合上面两幅图片，去理解下面的卷积计算过程：

![](https://img-blog.csdn.net/20160707204048899)

在上面的动图中，使用了两个卷积核 $\text{filter } \omega_0$ 和 $\text{filter } \omega_1$，分别得到了最右边 output 下的两个输出特征图，所以当前层使用了几个卷积核，就有几个输出特征图，即**当前卷积核的个数决定了当前层输出特征图的深度**。如果觉得某层模型的特征提取能力不足，可以在这一层多用几个卷积核提高这一层的特征提取能力。

#### 1.1.2感受野

感受野（Receptive Field）：卷积神经网络各输出特征图中的每个像素点，在原始输入图片上映射区域的大小。简单点说就是，输出特征图上的一个点对应输入图上的区域。有一张图来理解就是：

![](http://5b0988e595225.cdn.sohucs.com/images/20171130/7090489c708540ff9535d720e1666207.png)

根据上面这个图可以发现，对于一个 $5\times5$ 的输入特征图，经过两个 $3\times3$ 的卷积核的运算，可以得到一个像素点，那么对于这一个像素点来说，映射到输入特征图的感受野就是 5， 正好与输入特征图的大小一致。那么进一步再思考一下，如果对一个 $5\times5$ 的输入特征图，使用一个 $5\times5$ 的卷积核进行运算，得到也会是一个像素点，而对于这一个像素点来说它映射到输入特征图的感受野也是 5。

那么问题来了，既然对于一个相同的输入特征图，应用两个 $3\times3$ 的卷积核和应用一个 $5\times5$ 卷积核的特征提取能力是一样的，那么应该使用那种方案比较好？

这个时候就要考虑，两个方案所承载的待训练参数量和计算量了：
 + 对于两个 $3\times3$ 卷积核，参数量为 $3\times3 + 3\times3 = 18$ 个
 + 对于一个 $5\times5$ 卷积核，参数量为 $5 \times 5 = 25$ 个
 
在计算量上，假设输入特征图宽、高为 $x$，卷积核宽、高为 $k$，卷积计算步长为 1，输出图边长计算公式为 $输出图边长=(输入图边长-卷积核长+1)/步长$，那么输出特征图的像素个数就是输出图边长的平方，每个像素点都需要进行 $k^2$ 次乘加运算。因此总的计算量就是像素个数乘以乘加运算的次数：
 + 对于两个 $3\times3$ 卷积核，计算量为 $3\times3\times(x-3+1)^2 + 3\times3\times(x-3+1-3+1)^2=18x^2-108x+180$ 个
 + 对于一个 $5\times5$ 卷积核，计算量为 $5\times5(x-5+1)^2=25x^2-200x+400$ 个
 
直接给出结论，当 $x>10$ 时，两层 $3\times3$ 卷积核会比一个 $5\times5$ 卷积核的计算量少，也就是说前者会比后者的性能要好。这也是为什么现在的神经网络在卷积计算中常使用两层 $3\times3$ 卷积核替换一层 $5\times5$ 卷积核。

#### 1.1.3 填充

填充（padding）：为了保持输出图像尺寸与输入图像一致，经常会在输入图像周围进行全零填充。参考下面的动图：

![](http://img.zishu010.com/ArticleImage/2017/8/d658466e93f4488bbdfd0856bceb7b36.gif)

可以发现，输入特征图经过 $3\times$ 卷积核的运算后，得到的输出特征图的大小与输入特征图的大小一致，其原因就在于，在进行卷积运算之前，在输入特征图周围进行了全 0 填充。此外，是否使用全 0 填充，对输出特征图尺寸的大小也有影响：
$$
输出特征图边长=\begin{cases}
\lceil \frac{输入特征图边长}{步长} \rceil & \text{ if } padding=same(全0填充) \\ 
\lceil \frac{输入特征图边长-卷积核边长+1}{步长} \rceil & \text{ if } padding=valid(不全0填充) 
\end{cases}
$$

如果计算结果为小数，需要向上取整，比如计算结果若为 2.2，则结果应该变为 3。后面的条件 `padding=same` 和 `padding=valid` 就是与 TensorFlow 的 API 相对应，在使用 `tf.keras.layers.Conv2D()` 来设置卷积层时，对于参数 `padding` 的取值就是 `same` 或者 `valid`。

在 TensorFlow 中，使用 `tf.keras.layers.Conv2D` 来定义卷积层

```python
tf.keras.layers.Conv2D (
    filters=卷积核个数, 
    kernel_size=卷积核尺寸,   #正方形写核长整数，或（核高h，核宽w）
    strides=滑动步长,       #横纵向相同写步长整数，或(纵向步长h，横向步长w)，默认1
    padding='same'或'valid',  #使用全零填充是“same”，不使用是“valid”（默认）
    activation='relu'或'sigmoid'或'tanh'或'softmax'等 , #如卷积层之后还有批标准化操作，则不在这里使用激活函数
    input_shape=(高,宽,通道数) #输入特征图维度，可省略
)
```

### 1.2 批标准化层

神经网络对 0 附近的数据更敏感，但随着网络层数的增加，特征数据会出现偏离 0 均值的情况。标准化可以使数据符合以 0 为均值 1 为标准差的标准正态分布，把偏移的特征数据重新拉回到 0 附近。

批标准化（Batch Normalization，BN）就是对一小批（batch）数据进行标准化处理，使数据回归标准正态分布。常用在卷积操作和激活操作之间。

批标准化后，第 k 个卷积核的输出特征图中第 i 个像素点的值：

$$H_i^{'k}=\frac{H_i^k-\mu_{batch}^k}{\sigma_{batch}^k}$$

+ $H_i^k$：批标准化前，第 k 个卷积核，输出特征图中第 i 个像素点的值
+ $\mu_{batch}^k$：批标准化前，第 k 个卷积核，batch 张输出特征图中所有像素点平均值 $\mu_{batch}^k=\frac{1}{m}\sum_{i=1}^mH_i^k$
+ $\sigma_{batch}^k$：批标准化前，第 k 个卷积核，batch 张输出特征图中所有像素点标准差 $\sigma_{batch}^k=\sqrt{\delta+\frac{1}{m}\sum_{i=1}^m(H_i^k-\mu_{batch}^k)^2}$

BN 操作将原本偏移的特征数据重新拉回到 0 均值，使进入激活函数的数据分布在激活函数线性区，使得输入数据的微小变化，更明显的体现到激活函数的输出，提升了激活函数对输入数据的区分力。但是这种简单的特征数据标准化，使特征数据完全满足标准正态分布，集中在激活函数中心的线性区域。使激活函数丧失了非线性特性。因此在 BN 操作中为每个卷积核引入了两个可训练参数：缩放因子$\gamma$ 和 偏移因子$\beta$，在反向传播时会与其他参数一同被训练优化，使标准正态分布后的特征数据通过缩放因子和偏移因子优化了特征数据分布的宽窄和偏移量，保证了网络的非线性表达力。特征数据分布 $X_i^k$ 表示如下

$$X_i^k=\gamma H_i^{'k}+\beta_k$$

在 TensorFlow 中，使用 `tf.keras.layers.BatchNormalization()` 来表示批标准化层

### 1.3 池化层

池化（Pooling）操作用于减少卷积神经网络中的特征数量。主要法有最大池化和均值池化。最大值池化可以提取图片纹理，均值池化可以保留背景特征。如果用 $2\times2$ 的池化核对输入图片以 2 为步长进行池化，输出图片将变为输入图片的四分之一。

![](https://oscdn.geek-share.com/Uploads/Images/Content/201708/a18a8b34c940de778d27246fa6c3d950.jpeg)

最大值池化就是把与池化核覆盖区域的最大值输出，如下图（a）；均值池化就是把与池化核覆盖区域的均值输出，如下图（b）。

![](https://ss1.bdstatic.com/70cFvXSh_Q1YnxGkpoWK1HF6hhy/it/u=1901272252,3736359799&fm=26&gp=0.jpg)

在 Tensorflow 中分别使用 的是`tf.keras.layers.MaxPool2D` 函数和 `tf.keras.layers.AveragePooling2D` 函数，具体的使用方法如下： 

最大值池化
```python
tf.keras.layers.MaxPool2D( 
    pool_size=池化核尺寸,  #正方形写核长整数，或（核高h，核宽w）
    strides=池化步长,     #步长整数， 或(纵向步长h，横向步长w)，默认为pool_size
    padding='same'或'valid')#使用全零填充是“same”，不使用是“valid”（默认）
```

均值池化
```python
tf.keras.layers.AveragePooling2D( 
    pool_size=池化核尺寸,  #正方形写核长整数，或（核高h，核宽w）
    strides=池化步长,     #步长整数， 或(纵向步长h，横向步长w)，默认为pool_size
    padding='same'或'valid')#使用全零填充是“same”，不使用是“valid”（默认）
```

### 1.4 Dropout 层

为了缓解神经网络过拟合，在训练过程中，将一部分隐藏层的神经元按照一定比例从神经网络中临时舍弃，在使用时，再把舍弃的神经元恢复链接。如下图所示，图（a）为标准的全连接神经网络，图（b）为使用 dropout 后的网络：

![](https://img-blog.csdn.net/20180619185225799?watermark/2/text/aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3Byb2dyYW1fZGV2ZWxvcGVy/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)

在 TensorFlow 中，使用 `tf.keras.layers.Dropout(舍弃的概率)` 来表示 dropout 层。如果要临时舍弃 20% 的神经元，可以把舍弃的概率设为 0.2。

### 1.5 总结

以上简单的介绍了卷积神经网络基本操作步骤。卷积神经网络在完成特征提取后，就会把数据送入到全连接网络中，因此对于卷积神经网络的整个基本流程可以概括如下：

> 卷积 —》 批标准化 -》 激活 -》 池化 -》 Dropout -》 全连接
>
> **C**onvolutional —》 **B**atch Normal -》 **A**ctivation -》 **P**ooling -》 **D**ropout -》 全连接

说完了卷积神经网络，那如何用一句话总结“卷积”到底是什么呢？

说白了，**卷积就是特征提取器**，就是将数据送入全连接网络之前，对图像进行特征提取的操作。依据上面的执行流程，就是大写加粗的 **CBAPD**。

体现到用 TensorFlow 来搭建网络结构模型中，就如下所示：

```python
model = tf.keras.models.Sequential([
    Conv2D(filters=6, kernel_size=(5, 5), padding='same'), # 卷积层   C    
    BatchNormalization(),                       # BN 层    B
    Activation('relu'),                        # 激活层   A
    MaxPool2D(pool_size=(2, 2), strides=2, padding='same'),# 池化层   P
    Dropout(0.2),                            # Dropout层 D
])
```